In [55]:
from __future__ import print_function, division, absolute_import, unicode_literals

import numpy as np
import random
import math
import pickle
import os

import matlab.engine
from scipy.interpolate import interp1d

sigma = 10

plateau_min = -2
plateau_max = 2

r_min = 1
r_max = 200
bin_vector = [16, 30.8, 37.5, 45, 60]

width, height, cnt = [10, 10, 2]

num_cells = cnt

# Modified these paths
script_dir = os.path.dirname(os.getcwd())  #<-- absolute dir the script is in
rel_path = 'tf_unet/Attenuation_with_air_0_1.pkl'
abs_file_path = os.path.join(script_dir, rel_path)

# This is the part that imports the files
with open(abs_file_path, 'rb') as f:
    Attenuation, Attenuation2, Spec, Air = pickle.load(f)
Ev = Spec[:, 0]
Int = Spec[:, 1]

# f4 = interp1d(Ev, Int,kind='quadratic')
# Ev_long = np.linspace(Ev[0], Ev[-1], len(Ev) * 5 - 4)
# Int_long = f4(Ev_long)

Int_long = Int
Ev_long = Ev

Int_long = Int_long * 10000 / sum(Int_long)
#Convert the bin vector from energies to indices
nbins = len(bin_vector) - 1

for ii in range(nbins + 1):
    bin_vector[ii] = min(
        range(len(Ev_long)), key=lambda i: abs(Ev_long[i] - bin_vector[ii]))

image = np.zeros((width, height, nbins))
labels = np.zeros([width, height], dtype=np.bool)

imgx, imgy = width, height
nx = []  # hold the x value
ny = []  # holds the y value
nr = np.zeros([nbins, cnt])  # low energy image
nr2 = []  # high energy image
label = []  # labels whether or not the cell has cartilage

f1 = interp1d(np.log(Attenuation[12:, 0]),
              np.log(Attenuation[12:, 6]))  #,kind='quadratic')
f2 = interp1d(np.log(Attenuation2[19:, 0]),
              np.log(Attenuation2[19:, 6]))  #,kind='quadratic')
f3 = interp1d(np.log(Air[:, 0]), np.log(Air[:, 6]))  #,kind='quadratic')

target_SNR = 100
# Finding the singnal in water that will be the basis for the noise
signal = 3.1459 * 10**3
# Finding the target noise
sigma = signal / target_SNR
# Finding the signal strength that would give the correct poisson noise
correct_signal = (signal / sigma)**2.

for i in range(num_cells):

    # choosing random place for the cell
    nx.append(random.randrange(imgx))
    ny.append(random.randrange(imgy))
    rr = random.random()
    pp = random.random()

    # making a random value for the cartilage
    if i % 4 == 0:
        t_cart = random.uniform(0.1, 4)
        label.append(0)
    else:
        label.append(1)
        t_cart = 0.

    # Attenuating
    pixel_value = 0.

    #     for ll,t_total in enumerate([4,6,8]):
    t_cart = 4
    t_total = 4
    for jj in range(nbins):
        # finding the attenuation for the low image
        for ii, energy in enumerate(
                Ev_long[bin_vector[jj]:bin_vector[jj + 1]]):
            mu = f1(np.log(energy / 1000.))
            mu2 = f2(np.log(energy / 1000.))
            mu_air = f3(np.log(energy / 1000.))
            mu = np.exp(mu)
            mu2 = np.exp(mu2)
            mu_air = np.exp(mu_air)

            #                 plt.figure(1)
            #                 plt.plot(np.log(energy/1000.),np.log(mu),'r*')

            #mu = 0.2073
            # wrong density of water to match the image

            t_water = t_total - t_cart
            pixel_value += (Int_long[ii + bin_vector[jj]] * np.exp(
                -mu * t_water * 1.015) * np.exp(-mu2 * t_cart * 1.1) * np.exp(
                    -mu_air * 30 * 0.00120479))

#                 plt.figure(2)
#                 plt.plot(ii,pixel_value,'k*')


        nr[jj, i] = pixel_value
        pixel_value = 0.

for y in range(imgy):
    for x in range(imgx):
        dmin = math.hypot(imgx - 1, imgy - 1)
        j = -1
        for i in range(num_cells):
            d = math.hypot(nx[i] - x, ny[i] - y)
            if d < dmin:
                dmin = d
                j = i

        # Generating some poisson noise\
        for kk in range(nbins):
            image[x, y, kk] = nr[kk, j] + np.random.normal(0, np.sqrt(nr[kk, j]*.1))
            print(image[x, y, kk])
            #*correct_signal / signal + np.random.normal(
            #    0, np.sqrt(nr[kk, j] * correct_signal / signal))

        labels[x, y] = label[j]                                    

246.45368724487665
517.2379362302901
742.6078112204913
926.4973974751433
238.98867176405082
512.5566252095028
746.1975121653453
901.7808127854528
236.6857434094723
527.68060361787
724.8774504988688
920.8393190088207
239.0646132385385
515.6822533392681
731.2334929232945
941.8138472530652
230.6632543867523
508.4911387073666
736.6024155171251
917.8326024530251
246.5821793848336
503.1476341175346
750.0444337194507
909.5135945430404
225.62536572674387
518.8451674008203
752.9148791540899
917.9782293086413
233.1127985027223
518.3129680443612
734.0734699884696
917.6890739844216
232.89559205659197
516.3263556607378
735.6308494151378
927.5939335193863
234.08487859049146
522.3592205170132
762.2224501982565
909.315083510342
238.55194033390646
509.17585021186653
735.4349528511042
920.2827916550916
242.59995833536962
504.8069469093388
749.7477421553477
907.0483560759202
238.3604684372632
517.99806942026
752.8427285326221
911.4287506374718
231.44297885118317
523.3780487132294
737.048230753206
916.585

In [59]:
matlab_reshape = np.reshape(image,[image.size],order='F').copy()
print(matlab_reshape)
matlab_reshape = list(np.squeeze(matlab_reshape)).copy()
matlab_reshape = matlab.double(matlab_reshape)

eng = matlab.engine.start_matlab()
ys = eng.net_val(matlab_reshape)

[246.45368724 238.98867176 236.68574341 239.06461324 230.66325439
 246.58217938 225.62536573 233.1127985  232.89559206 234.08487859
 238.55194033 242.59995834 238.36046844 231.44297885 227.30043781
 239.59303393 235.9581821  236.64624435 240.68286573 237.26354712
 237.05474594 222.98987607 228.39604939 243.48112493 235.95221947
 233.44671115 234.12395773 236.87307548 231.31443752 238.54077382
 230.04547738 231.88253093 227.41954028 242.98459901 239.16032896
 231.02870903 238.80126506 236.72610453 243.62684693 228.17834628
 249.25833144 234.14949931 237.56027634 236.93547231 238.46990763
 239.90288086 238.79060301 232.84966684 231.48146284 249.34858611
 236.1033285  233.11373896 237.24098859 234.5477397  238.83895802
 239.95704853 238.49883552 234.80970855 237.28840581 234.41099042
 234.58358533 236.55857079 232.77422077 234.7884746  233.01265179
 234.14626143 239.89925851 229.55905439 240.74640382 240.51485734
 222.4937214  232.95899602 235.34984736 232.81487984 239.75505557
 237.51147

In [61]:
im_cart = np.reshape(ys,[width,height],order='F').copy()
#return image, labels

%matplotlib qt
import matplotlib.pyplot as plt

plt.imshow(im_cart[:,:])
plt.figure()
plt.imshow(image[:,:,1])
plt.figure()
plt.imshow(labels)

(1, 300)

In [46]:
np.size(matlab_reshape)

(1, 1048576)

In [47]:
512*512*4

1048576

In [48]:
1048576/262144

4.0

In [52]:
import scipy.io as sio
sio.savemat("images.mat", {"image":image})